In [1]:
#All the import statements goes here
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import matplotlib as plt
import os

path1 = "J:/SF/Thermal Load Prediction/Repos folders/loadPrediction/"
path_list = ["J:/SF/Thermal Load Prediction/Datas/Open Power System Data/opsd-household_data-2017-09-01/",
             "/Volumes/ADITHYA/Adithya/ML materials/DATA SETS/Open Power System /opsd-household_data-2017-09-01/"]

if("adithya" in os.getlogin()):
    path = path_list[1]
else :
    path = path_list[0]

Using TensorFlow backend.


In [2]:
# read1 contains all rows and columns from csv file
read1 = pd.read_csv(path+"household_data_1min_singleindex.csv")

# read will have columns with load data
read = read1[read1.columns[2:11]]
x = np.array(read.values)

# t is to find out which minute in the data has minimum occurances of NaN
t = np.zeros(shape=(60))

for i in range(60):
    j=i;
    c=0
    while(j <= x.shape[0]-1):
        c1=0
        for k in x[j]:
            if(np.isnan(k)):
                c1+=1
        if(c1!=0):
            c+=1
        j+=60
    t[i]=c
    
# amin(t) gives minimum count and argmin(t)+1 gives minute value
print(np.amin(t))
print(np.argmin(t)+1)

1290.0
10


In [3]:
# using 1hr data
hr_data = pd.read_csv(path+"household_data_60min_singleindex.csv")
# removing the following columns
del hr_data["interpolated_values"]
del hr_data["DE_KN_residential4_grid_export"]
del hr_data["DE_KN_residential4_pv"]
del hr_data["cet_cest_timestamp"]

# yr_data will have the data from 2016-01-01 to 2017-01-01
yr_data = pd.DataFrame(hr_data.values[2137:10921])
yr_data.columns = hr_data.columns

In [4]:
# doing linear extrapolation on data ..ie.. if x is notNull and y in notNull and we have a set of nulls between them,
# we do : 
# gradient = (y-x)/(no. of nulls)
# add gradient to values from x to y recursively so that we fill the gap
df = pd.DataFrame()
for i in range(len(yr_data.columns)):
    if(i != 0):
        df = yr_data.iloc[:,i]
        flag1 = False
        for j in range(df.values.shape[0]):
            if(np.isnan(df.values[j]) and not(flag1)):
                prev = j-1
                flag1 = True
            elif(flag1 and not(np.isnan(df.values[j]))):
                grad = (df.values[j] - df.values[prev])/(j-prev)
                for k in range(prev+1, j):
                    df.values[k] = df.values[k-1] + grad
                flag1 = False
        yr_data[yr_data.columns[i]] = df.values

In [5]:
# exporting the new non null containing data to new csv
yr_data.to_csv("Year_Data_New.csv",sep=",",encoding="utf-8")

In [6]:
# doing linear extrapolation on data ..ie.. if x is notNull and y in notNull and we have a set of nulls between them,
# we do : 
# gradient = (y-x)/(no. of nulls)
# add gradient to values from x to y recursively so that we fill the gap on hr data
df = pd.DataFrame()
for i in range(len(hr_data.columns)):
    if(i != 0):
        df = hr_data.iloc[:,i]
        flag1 = False
        flag2 = False
        for j in range(df.values.shape[0]):
            if(np.isnan(df.values[j]) and not(flag1)):
                if(j==0):
                    flag2 = True
                prev = j-1
                flag1 = True
            elif(flag1 and not(np.isnan(df.values[j]))):
                if(flag2):
                    flag2 = False
                    for k in range(prev+1,j):
                        df.values[k] = df.values[j]
                else:
                    grad = (df.values[j] - df.values[prev])/(j-prev)
                    for k in range(prev+1, j):
                        df.values[k] = df.values[k-1] + grad
                flag1 = False
        hr_data[hr_data.columns[i]] = df.values

In [7]:
# counted null values after interpolation in entire dataset
hr_data = hr_data.iloc[:11012,:]

nan_cnt = []

nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_dishwasher'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_ev'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_freezer'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_grid_import'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_heat_pump'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_refrigerator'])))
nan_cnt.append(sum(pd.isnull(hr_data['DE_KN_residential4_washing_machine'])))

print(nan_cnt)

[0, 0, 0, 0, 0, 0, 0]


In [8]:
hr_data.to_csv("Hour_Data_New.csv",sep=",",encoding="utf-8")
hr_data = pd.read_csv(path1+"Hour_Data_New.csv")
del hr_data['Unnamed: 0']

In [29]:
# relu activation for input layer alone.. output layer softmax made it worser..
# this is on entire dataset till first row with null values after interpolation
for i in range(len(hr_data.columns)):
    df = pd.DataFrame()
    if(i != 0):
        print(hr_data.columns[i])
        df = hr_data[hr_data.columns[i]]
        ndf = pd.concat([df.shift(2), df.shift(1), df], axis=1)
        ndf.columns = ('t-2', 't-1', 't')
        ndf = ndf.iloc[2:]
        
        x = ndf[ndf.columns[0:2]]
        y = ndf[ndf.columns[-1:]]
        
        X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)
        model = Sequential()
        model.add(Dense(5,input_dim=2,kernel_initializer='normal', activation='relu'))
        #Output Layer.
        model.add(Dense(1,kernel_initializer='normal'))
        #Compile model.
        model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
        #Fit the model.
        model.fit(X_train,y_train,epochs=300,batch_size=200,verbose=2)
        #Final Evaluation of the model.
        scores = model.evaluate(X_test,y_test,verbose=0)
        print ("Baseline Error : %.5f" % (100-scores[1]*100))
        model.summary()

DE_KN_residential4_dishwasher
Epoch 1/300
 - 1s - loss: 4564.4624 - mean_squared_error: 4564.4624
Epoch 2/300
 - 0s - loss: 4190.6533 - mean_squared_error: 4190.6533
Epoch 3/300
 - 0s - loss: 3551.9152 - mean_squared_error: 3551.9152
Epoch 4/300
 - 0s - loss: 2716.6273 - mean_squared_error: 2716.6273
Epoch 5/300
 - 0s - loss: 1842.5522 - mean_squared_error: 1842.5522
Epoch 6/300
 - 0s - loss: 1074.7182 - mean_squared_error: 1074.7182
Epoch 7/300
 - 0s - loss: 524.8760 - mean_squared_error: 524.8760
Epoch 8/300
 - 0s - loss: 208.3675 - mean_squared_error: 208.3675
Epoch 9/300
 - 0s - loss: 66.3150 - mean_squared_error: 66.3150
Epoch 10/300
 - 0s - loss: 17.0550 - mean_squared_error: 17.0550
Epoch 11/300
 - 0s - loss: 3.6408 - mean_squared_error: 3.6408
Epoch 12/300
 - 0s - loss: 0.7396 - mean_squared_error: 0.7396
Epoch 13/300
 - 0s - loss: 0.2139 - mean_squared_error: 0.2139
Epoch 14/300
 - 0s - loss: 0.1375 - mean_squared_error: 0.1375
Epoch 15/300
 - 0s - loss: 0.1281 - mean_squared_

Epoch 130/300
 - 0s - loss: 0.0260 - mean_squared_error: 0.0260
Epoch 131/300
 - 0s - loss: 0.0251 - mean_squared_error: 0.0251
Epoch 132/300
 - 0s - loss: 0.0243 - mean_squared_error: 0.0243
Epoch 133/300
 - 0s - loss: 0.0235 - mean_squared_error: 0.0235
Epoch 134/300
 - 0s - loss: 0.0227 - mean_squared_error: 0.0227
Epoch 135/300
 - 0s - loss: 0.0220 - mean_squared_error: 0.0220
Epoch 136/300
 - 0s - loss: 0.0212 - mean_squared_error: 0.0212
Epoch 137/300
 - 0s - loss: 0.0205 - mean_squared_error: 0.0205
Epoch 138/300
 - 0s - loss: 0.0198 - mean_squared_error: 0.0198
Epoch 139/300
 - 0s - loss: 0.0191 - mean_squared_error: 0.0191
Epoch 140/300
 - 0s - loss: 0.0184 - mean_squared_error: 0.0184
Epoch 141/300
 - 0s - loss: 0.0178 - mean_squared_error: 0.0178
Epoch 142/300
 - 0s - loss: 0.0172 - mean_squared_error: 0.0172
Epoch 143/300
 - 0s - loss: 0.0166 - mean_squared_error: 0.0166
Epoch 144/300
 - 0s - loss: 0.0160 - mean_squared_error: 0.0160
Epoch 145/300
 - 0s - loss: 0.0155 - mea

 - 0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 259/300
 - 0s - loss: 0.0075 - mean_squared_error: 0.0075
Epoch 260/300
 - 0s - loss: 0.0076 - mean_squared_error: 0.0076
Epoch 261/300
 - 0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 262/300
 - 0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 263/300
 - 0s - loss: 0.0073 - mean_squared_error: 0.0073
Epoch 264/300
 - 0s - loss: 0.0073 - mean_squared_error: 0.0073
Epoch 265/300
 - 0s - loss: 0.0075 - mean_squared_error: 0.0075
Epoch 266/300
 - 0s - loss: 0.0075 - mean_squared_error: 0.0075
Epoch 267/300
 - 0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 268/300
 - 0s - loss: 0.0074 - mean_squared_error: 0.0074
Epoch 269/300
 - 0s - loss: 0.0076 - mean_squared_error: 0.0076
Epoch 270/300
 - 0s - loss: 0.0105 - mean_squared_error: 0.0105
Epoch 271/300
 - 0s - loss: 0.0075 - mean_squared_error: 0.0075
Epoch 272/300
 - 0s - loss: 0.0073 - mean_squared_error: 0.0073
Epoch 273/300
 - 0s - loss: 0.0075 - mean_squared_erro

 - 0s - loss: 0.4370 - mean_squared_error: 0.4370
Epoch 77/300
 - 0s - loss: 0.4368 - mean_squared_error: 0.4368
Epoch 78/300
 - 0s - loss: 0.4367 - mean_squared_error: 0.4367
Epoch 79/300
 - 0s - loss: 0.4364 - mean_squared_error: 0.4364
Epoch 80/300
 - 0s - loss: 0.4363 - mean_squared_error: 0.4363
Epoch 81/300
 - 0s - loss: 0.4361 - mean_squared_error: 0.4361
Epoch 82/300
 - 0s - loss: 0.4359 - mean_squared_error: 0.4359
Epoch 83/300
 - 0s - loss: 0.4356 - mean_squared_error: 0.4356
Epoch 84/300
 - 0s - loss: 0.4355 - mean_squared_error: 0.4355
Epoch 85/300
 - 0s - loss: 0.4353 - mean_squared_error: 0.4353
Epoch 86/300
 - 0s - loss: 0.4350 - mean_squared_error: 0.4350
Epoch 87/300
 - 0s - loss: 0.4350 - mean_squared_error: 0.4350
Epoch 88/300
 - 0s - loss: 0.4346 - mean_squared_error: 0.4346
Epoch 89/300
 - 0s - loss: 0.4346 - mean_squared_error: 0.4346
Epoch 90/300
 - 0s - loss: 0.4341 - mean_squared_error: 0.4341
Epoch 91/300
 - 0s - loss: 0.4339 - mean_squared_error: 0.4339
Epoch

Epoch 205/300
 - 0s - loss: 0.3654 - mean_squared_error: 0.3654
Epoch 206/300
 - 0s - loss: 0.3680 - mean_squared_error: 0.3680
Epoch 207/300
 - 0s - loss: 0.3629 - mean_squared_error: 0.3629
Epoch 208/300
 - 0s - loss: 0.3640 - mean_squared_error: 0.3640
Epoch 209/300
 - 0s - loss: 0.3636 - mean_squared_error: 0.3636
Epoch 210/300
 - 0s - loss: 0.3617 - mean_squared_error: 0.3617
Epoch 211/300
 - 0s - loss: 0.3644 - mean_squared_error: 0.3644
Epoch 212/300
 - 0s - loss: 0.3619 - mean_squared_error: 0.3619
Epoch 213/300
 - 0s - loss: 0.3620 - mean_squared_error: 0.3620
Epoch 214/300
 - 0s - loss: 0.3575 - mean_squared_error: 0.3575
Epoch 215/300
 - 0s - loss: 0.3559 - mean_squared_error: 0.3559
Epoch 216/300
 - 0s - loss: 0.3561 - mean_squared_error: 0.3561
Epoch 217/300
 - 0s - loss: 0.3549 - mean_squared_error: 0.3549
Epoch 218/300
 - 0s - loss: 0.3570 - mean_squared_error: 0.3570
Epoch 219/300
 - 0s - loss: 0.3598 - mean_squared_error: 0.3598
Epoch 220/300
 - 0s - loss: 0.3609 - mea

...

Epoch 96/300
 - 0s - loss: 2.1014e-04 - mean_squared_error: 2.1014e-04
Epoch 97/300
 - 0s - loss: 1.9748e-04 - mean_squared_error: 1.9748e-04
Epoch 98/300
 - 0s - loss: 1.8512e-04 - mean_squared_error: 1.8512e-04
Epoch 99/300
 - 0s - loss: 2.2452e-04 - mean_squared_error: 2.2452e-04
Epoch 100/300
 - 0s - loss: 3.0273e-04 - mean_squared_error: 3.0273e-04
Epoch 101/300
 - 0s - loss: 2.1297e-04 - mean_squared_error: 2.1297e-04
Epoch 102/300
 - 0s - loss: 2.0267e-04 - mean_squared_error: 2.0267e-04
Epoch 103/300
 - 0s - loss: 3.3745e-04 - mean_squared_error: 3.3745e-04
Epoch 104/300
 - 0s - loss: 2.1540e-04 - mean_squared_error: 2.1540e-04
Epoch 105/300
 - 0s - loss: 1.9275e-04 - mean_squared_error: 1.9275e-04
Epoch 106/300
 - 0s - loss: 2.3374e-04 - mean_squared_error: 2.3374e-04
Epoch 107/300
 - 0s - loss: 1.8594e-04 - mean_squared_error: 1.8594e-04
Epoch 108/300
 - 0s - loss: 1.9032e-04 - mean_squared_error: 1.9032e-04
Epoch 109/300
 - 0s - loss: 1.9861e-04 - mean_squared_error: 1.9861e

Epoch 210/300
 - 0s - loss: 2.1161e-04 - mean_squared_error: 2.1161e-04
Epoch 211/300
 - 0s - loss: 1.8132e-04 - mean_squared_error: 1.8132e-04
Epoch 212/300
 - 0s - loss: 1.8599e-04 - mean_squared_error: 1.8599e-04
Epoch 213/300
 - 0s - loss: 1.9390e-04 - mean_squared_error: 1.9390e-04
Epoch 214/300
 - 0s - loss: 2.6786e-04 - mean_squared_error: 2.6786e-04
Epoch 215/300
 - 0s - loss: 2.0459e-04 - mean_squared_error: 2.0459e-04
Epoch 216/300
 - 0s - loss: 1.9593e-04 - mean_squared_error: 1.9593e-04
Epoch 217/300
 - 0s - loss: 2.1826e-04 - mean_squared_error: 2.1826e-04
Epoch 218/300
 - 0s - loss: 2.2102e-04 - mean_squared_error: 2.2102e-04
Epoch 219/300
 - 0s - loss: 2.4374e-04 - mean_squared_error: 2.4374e-04
Epoch 220/300
 - 0s - loss: 4.6578e-04 - mean_squared_error: 4.6578e-04
Epoch 221/300
 - 0s - loss: 2.4322e-04 - mean_squared_error: 2.4322e-04
Epoch 222/300
 - 0s - loss: 4.4287e-04 - mean_squared_error: 4.4287e-04
Epoch 223/300
 - 0s - loss: 6.1144e-04 - mean_squared_error: 6.1

Epoch 16/300
 - 0s - loss: 2.5863 - mean_squared_error: 2.5863
Epoch 17/300
 - 0s - loss: 0.9106 - mean_squared_error: 0.9106
Epoch 18/300
 - 0s - loss: 0.4101 - mean_squared_error: 0.4101
Epoch 19/300
 - 0s - loss: 0.2689 - mean_squared_error: 0.2689
Epoch 20/300
 - 0s - loss: 0.2320 - mean_squared_error: 0.2320
Epoch 21/300
 - 0s - loss: 0.2235 - mean_squared_error: 0.2235
Epoch 22/300
 - 0s - loss: 0.2212 - mean_squared_error: 0.2212
Epoch 23/300
 - 0s - loss: 0.2203 - mean_squared_error: 0.2203
Epoch 24/300
 - 0s - loss: 0.2196 - mean_squared_error: 0.2196
Epoch 25/300
 - 0s - loss: 0.2191 - mean_squared_error: 0.2191
Epoch 26/300
 - 0s - loss: 0.2185 - mean_squared_error: 0.2185
Epoch 27/300
 - 0s - loss: 0.2179 - mean_squared_error: 0.2179
Epoch 28/300
 - 0s - loss: 0.2173 - mean_squared_error: 0.2173
Epoch 29/300
 - 0s - loss: 0.2167 - mean_squared_error: 0.2167
Epoch 30/300
 - 0s - loss: 0.2161 - mean_squared_error: 0.2161
Epoch 31/300
 - 0s - loss: 0.2154 - mean_squared_error:

Epoch 146/300
 - 0s - loss: 0.0373 - mean_squared_error: 0.0373
Epoch 147/300
 - 0s - loss: 0.0359 - mean_squared_error: 0.0359
Epoch 148/300
 - 0s - loss: 0.0346 - mean_squared_error: 0.0346
Epoch 149/300
 - 0s - loss: 0.0334 - mean_squared_error: 0.0334
Epoch 150/300
 - 0s - loss: 0.0321 - mean_squared_error: 0.0321
Epoch 151/300
 - 0s - loss: 0.0309 - mean_squared_error: 0.0309
Epoch 152/300
 - 0s - loss: 0.0298 - mean_squared_error: 0.0298
Epoch 153/300
 - 0s - loss: 0.0287 - mean_squared_error: 0.0287
Epoch 154/300
 - 0s - loss: 0.0276 - mean_squared_error: 0.0276
Epoch 155/300
 - 0s - loss: 0.0265 - mean_squared_error: 0.0265
Epoch 156/300
 - 0s - loss: 0.0255 - mean_squared_error: 0.0255
Epoch 157/300
 - 0s - loss: 0.0246 - mean_squared_error: 0.0246
Epoch 158/300
 - 0s - loss: 0.0236 - mean_squared_error: 0.0236
Epoch 159/300
 - 0s - loss: 0.0227 - mean_squared_error: 0.0227
Epoch 160/300
 - 0s - loss: 0.0219 - mean_squared_error: 0.0219
Epoch 161/300
 - 0s - loss: 0.0211 - mea

 - 0s - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 275/300
 - 0s - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 276/300
 - 0s - loss: 0.0100 - mean_squared_error: 0.0100
Epoch 277/300
 - 0s - loss: 0.0095 - mean_squared_error: 0.0095
Epoch 278/300
 - 0s - loss: 0.0095 - mean_squared_error: 0.0095
Epoch 279/300
 - 0s - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 280/300
 - 0s - loss: 0.0095 - mean_squared_error: 0.0095
Epoch 281/300
 - 0s - loss: 0.0100 - mean_squared_error: 0.0100
Epoch 282/300
 - 0s - loss: 0.0095 - mean_squared_error: 0.0095
Epoch 283/300
 - 0s - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 284/300
 - 0s - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 285/300
 - 0s - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 286/300
 - 0s - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 287/300
 - 0s - loss: 0.0105 - mean_squared_error: 0.0105
Epoch 288/300
 - 0s - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 289/300
 - 0s - loss: 0.0096 - mean_squared_erro

In [32]:
# relu activation for input layer alone.. output layer softmax made it worser..
# this is on the year 2016-01-01 to 2017-01-01 data
for i in range(len(yr_data.columns)):
    df = pd.DataFrame()
    if(i != 0):
        print(yr_data.columns[i])
        df = yr_data[yr_data.columns[i]]
        ndf = pd.concat([df.shift(2), df.shift(1), df], axis=1)
        ndf.columns = ('t-2', 't-1', 't')
        ndf = ndf.iloc[2:]
        
        x = ndf[ndf.columns[0:2]]
        y = ndf[ndf.columns[-1:]]
        
        X_train,X_test,y_train,y_test = train_test_split(x.values,y.values,test_size=0.2,random_state=1)
        model = Sequential()
        model.add(Dense(5,input_dim=2,kernel_initializer='normal', activation='relu'))
        #Output Layer.
        model.add(Dense(1,kernel_initializer='normal'))
        #Compile model.
        model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mean_squared_error'])
        #Fit the model.
        model.fit(X_train,y_train,epochs=200,batch_size=200,verbose=2)
        #Final Evaluation of the model.
        scores = model.evaluate(X_test,y_test,verbose=0)
        print ("Baseline Error : %.5f" % (100-scores[1]*100))
        model.summary()

DE_KN_residential4_dishwasher
Epoch 1/200
 - 2s - loss: 5724.1163 - mean_squared_error: 5724.1163
Epoch 2/200
 - 0s - loss: 5546.2395 - mean_squared_error: 5546.2395
Epoch 3/200
 - 0s - loss: 5286.4263 - mean_squared_error: 5286.4263
Epoch 4/200
 - 0s - loss: 4901.2085 - mean_squared_error: 4901.2085
Epoch 5/200
 - 0s - loss: 4402.8216 - mean_squared_error: 4402.8216
Epoch 6/200
 - 0s - loss: 3825.5237 - mean_squared_error: 3825.5237
Epoch 7/200
 - 0s - loss: 3199.9748 - mean_squared_error: 3199.9748
Epoch 8/200
 - 0s - loss: 2569.3036 - mean_squared_error: 2569.3036
Epoch 9/200
 - 0s - loss: 1972.0704 - mean_squared_error: 1972.0704
Epoch 10/200
 - 0s - loss: 1440.0719 - mean_squared_error: 1440.0719
Epoch 11/200
 - 0s - loss: 996.8910 - mean_squared_error: 996.8910
Epoch 12/200
 - 0s - loss: 648.9037 - mean_squared_error: 648.9037
Epoch 13/200
 - 0s - loss: 397.1137 - mean_squared_error: 397.1137
Epoch 14/200
 - 0s - loss: 227.3153 - mean_squared_error: 227.3153
Epoch 15/200
 - 0s - 

Epoch 129/200
 - 0s - loss: 0.0658 - mean_squared_error: 0.0658
Epoch 130/200
 - 0s - loss: 0.0656 - mean_squared_error: 0.0656
Epoch 131/200
 - 0s - loss: 0.0653 - mean_squared_error: 0.0653
Epoch 132/200
 - 0s - loss: 0.0650 - mean_squared_error: 0.0650
Epoch 133/200
 - 0s - loss: 0.0647 - mean_squared_error: 0.0647
Epoch 134/200
 - 0s - loss: 0.0644 - mean_squared_error: 0.0644
Epoch 135/200
 - 0s - loss: 0.0641 - mean_squared_error: 0.0641
Epoch 136/200
 - 0s - loss: 0.0638 - mean_squared_error: 0.0638
Epoch 137/200
 - 0s - loss: 0.0635 - mean_squared_error: 0.0635
Epoch 138/200
 - 0s - loss: 0.0632 - mean_squared_error: 0.0632
Epoch 139/200
 - 0s - loss: 0.0629 - mean_squared_error: 0.0629
Epoch 140/200
 - 0s - loss: 0.0626 - mean_squared_error: 0.0626
Epoch 141/200
 - 0s - loss: 0.0622 - mean_squared_error: 0.0622
Epoch 142/200
 - 0s - loss: 0.0619 - mean_squared_error: 0.0619
Epoch 143/200
 - 0s - loss: 0.0616 - mean_squared_error: 0.0616
Epoch 144/200
 - 0s - loss: 0.0613 - mea

Epoch 47/200
 - 0s - loss: 0.3761 - mean_squared_error: 0.3761
Epoch 48/200
 - 0s - loss: 0.3762 - mean_squared_error: 0.3762
Epoch 49/200
 - 0s - loss: 0.3763 - mean_squared_error: 0.3763
Epoch 50/200
 - 0s - loss: 0.3762 - mean_squared_error: 0.3762
Epoch 51/200
 - 0s - loss: 0.3761 - mean_squared_error: 0.3761
Epoch 52/200
 - 0s - loss: 0.3761 - mean_squared_error: 0.3761
Epoch 53/200
 - 0s - loss: 0.3761 - mean_squared_error: 0.3761
Epoch 54/200
 - 0s - loss: 0.3761 - mean_squared_error: 0.3761
Epoch 55/200
 - 0s - loss: 0.3761 - mean_squared_error: 0.3761
Epoch 56/200
 - 0s - loss: 0.3760 - mean_squared_error: 0.3760
Epoch 57/200
 - 0s - loss: 0.3760 - mean_squared_error: 0.3760
Epoch 58/200
 - 0s - loss: 0.3760 - mean_squared_error: 0.3760
Epoch 59/200
 - 0s - loss: 0.3761 - mean_squared_error: 0.3761
Epoch 60/200
 - 0s - loss: 0.3760 - mean_squared_error: 0.3760
Epoch 61/200
 - 0s - loss: 0.3760 - mean_squared_error: 0.3760
Epoch 62/200
 - 0s - loss: 0.3759 - mean_squared_error:

Epoch 176/200
 - 0s - loss: 0.3701 - mean_squared_error: 0.3701
Epoch 177/200
 - 0s - loss: 0.3701 - mean_squared_error: 0.3701
Epoch 178/200
 - 0s - loss: 0.3700 - mean_squared_error: 0.3700
Epoch 179/200
 - 0s - loss: 0.3701 - mean_squared_error: 0.3701
Epoch 180/200
 - 0s - loss: 0.3705 - mean_squared_error: 0.3705
Epoch 181/200
 - 0s - loss: 0.3698 - mean_squared_error: 0.3698
Epoch 182/200
 - 0s - loss: 0.3700 - mean_squared_error: 0.3700
Epoch 183/200
 - 0s - loss: 0.3693 - mean_squared_error: 0.3693
Epoch 184/200
 - 0s - loss: 0.3701 - mean_squared_error: 0.3701
Epoch 185/200
 - 0s - loss: 0.3694 - mean_squared_error: 0.3694
Epoch 186/200
 - 0s - loss: 0.3691 - mean_squared_error: 0.3691
Epoch 187/200
 - 0s - loss: 0.3692 - mean_squared_error: 0.3692
Epoch 188/200
 - 0s - loss: 0.3685 - mean_squared_error: 0.3685
Epoch 189/200
 - 0s - loss: 0.3692 - mean_squared_error: 0.3692
Epoch 190/200
 - 0s - loss: 0.3687 - mean_squared_error: 0.3687
Epoch 191/200
 - 0s - loss: 0.3682 - mea

Epoch 95/200
 - 0s - loss: 0.0589 - mean_squared_error: 0.0589
Epoch 96/200
 - 0s - loss: 0.0586 - mean_squared_error: 0.0586
Epoch 97/200
 - 0s - loss: 0.0583 - mean_squared_error: 0.0583
Epoch 98/200
 - 0s - loss: 0.0580 - mean_squared_error: 0.0580
Epoch 99/200
 - 0s - loss: 0.0577 - mean_squared_error: 0.0577
Epoch 100/200
 - 0s - loss: 0.0574 - mean_squared_error: 0.0574
Epoch 101/200
 - 0s - loss: 0.0571 - mean_squared_error: 0.0571
Epoch 102/200
 - 0s - loss: 0.0568 - mean_squared_error: 0.0568
Epoch 103/200
 - 0s - loss: 0.0564 - mean_squared_error: 0.0564
Epoch 104/200
 - 0s - loss: 0.0561 - mean_squared_error: 0.0561
Epoch 105/200
 - 0s - loss: 0.0558 - mean_squared_error: 0.0558
Epoch 106/200
 - 0s - loss: 0.0554 - mean_squared_error: 0.0554
Epoch 107/200
 - 0s - loss: 0.0551 - mean_squared_error: 0.0551
Epoch 108/200
 - 0s - loss: 0.0548 - mean_squared_error: 0.0548
Epoch 109/200
 - 0s - loss: 0.0544 - mean_squared_error: 0.0544
Epoch 110/200
 - 0s - loss: 0.0541 - mean_squ

...

Epoch 59/200
 - 0s - loss: 0.2982 - mean_squared_error: 0.2982
Epoch 60/200
 - 0s - loss: 0.2981 - mean_squared_error: 0.2981
Epoch 61/200
 - 0s - loss: 0.2982 - mean_squared_error: 0.2982
Epoch 62/200
 - 0s - loss: 0.2981 - mean_squared_error: 0.2981
Epoch 63/200
 - 0s - loss: 0.2982 - mean_squared_error: 0.2982
Epoch 64/200
 - 0s - loss: 0.2982 - mean_squared_error: 0.2982
Epoch 65/200
 - 0s - loss: 0.2983 - mean_squared_error: 0.2983
Epoch 66/200
 - 0s - loss: 0.2983 - mean_squared_error: 0.2983
Epoch 67/200
 - 0s - loss: 0.2983 - mean_squared_error: 0.2983
Epoch 68/200
 - 0s - loss: 0.2982 - mean_squared_error: 0.2982
Epoch 69/200
 - 0s - loss: 0.2982 - mean_squared_error: 0.2982
Epoch 70/200
 - 0s - loss: 0.2983 - mean_squared_error: 0.2983
Epoch 71/200
 - 0s - loss: 0.2983 - mean_squared_error: 0.2983
Epoch 72/200
 - 0s - loss: 0.2982 - mean_squared_error: 0.2982
Epoch 73/200
 - 0s - loss: 0.2983 - mean_squared_error: 0.2983
Epoch 74/200
 - 0s - loss: 0.2983 - mean_squared_error:

Epoch 188/200
 - 0s - loss: 0.2992 - mean_squared_error: 0.2992
Epoch 189/200
 - 0s - loss: 0.3002 - mean_squared_error: 0.3002
Epoch 190/200
 - 0s - loss: 0.2985 - mean_squared_error: 0.2985
Epoch 191/200
 - 0s - loss: 0.2993 - mean_squared_error: 0.2993
Epoch 192/200
 - 0s - loss: 0.2987 - mean_squared_error: 0.2987
Epoch 193/200
 - 0s - loss: 0.2987 - mean_squared_error: 0.2987
Epoch 194/200
 - 0s - loss: 0.3005 - mean_squared_error: 0.3005
Epoch 195/200
 - 0s - loss: 0.3001 - mean_squared_error: 0.3001
Epoch 196/200
 - 0s - loss: 0.3004 - mean_squared_error: 0.3004
Epoch 197/200
 - 0s - loss: 0.2991 - mean_squared_error: 0.2991
Epoch 198/200
 - 0s - loss: 0.2987 - mean_squared_error: 0.2987
Epoch 199/200
 - 0s - loss: 0.2990 - mean_squared_error: 0.2990
Epoch 200/200
 - 0s - loss: 0.2997 - mean_squared_error: 0.2997
Baseline Error : 70.04238
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_157 

Epoch 107/200
 - 0s - loss: 0.0593 - mean_squared_error: 0.0593
Epoch 108/200
 - 0s - loss: 0.0588 - mean_squared_error: 0.0588
Epoch 109/200
 - 0s - loss: 0.0584 - mean_squared_error: 0.0584
Epoch 110/200
 - 0s - loss: 0.0579 - mean_squared_error: 0.0579
Epoch 111/200
 - 0s - loss: 0.0575 - mean_squared_error: 0.0575
Epoch 112/200
 - 0s - loss: 0.0570 - mean_squared_error: 0.0570
Epoch 113/200
 - 0s - loss: 0.0565 - mean_squared_error: 0.0565
Epoch 114/200
 - 0s - loss: 0.0560 - mean_squared_error: 0.0560
Epoch 115/200
 - 0s - loss: 0.0555 - mean_squared_error: 0.0555
Epoch 116/200
 - 0s - loss: 0.0551 - mean_squared_error: 0.0551
Epoch 117/200
 - 0s - loss: 0.0545 - mean_squared_error: 0.0545
Epoch 118/200
 - 0s - loss: 0.0540 - mean_squared_error: 0.0540
Epoch 119/200
 - 0s - loss: 0.0535 - mean_squared_error: 0.0535
Epoch 120/200
 - 0s - loss: 0.0530 - mean_squared_error: 0.0530
Epoch 121/200
 - 0s - loss: 0.0525 - mean_squared_error: 0.0525
Epoch 122/200
 - 0s - loss: 0.0520 - mea

Epoch 24/200
 - 0s - loss: 0.1537 - mean_squared_error: 0.1537
Epoch 25/200
 - 0s - loss: 0.1312 - mean_squared_error: 0.1312
Epoch 26/200
 - 0s - loss: 0.1241 - mean_squared_error: 0.1241
Epoch 27/200
 - 0s - loss: 0.1220 - mean_squared_error: 0.1220
Epoch 28/200
 - 0s - loss: 0.1213 - mean_squared_error: 0.1213
Epoch 29/200
 - 0s - loss: 0.1211 - mean_squared_error: 0.1211
Epoch 30/200
 - 0s - loss: 0.1209 - mean_squared_error: 0.1209
Epoch 31/200
 - 0s - loss: 0.1208 - mean_squared_error: 0.1208
Epoch 32/200
 - 0s - loss: 0.1207 - mean_squared_error: 0.1207
Epoch 33/200
 - 0s - loss: 0.1206 - mean_squared_error: 0.1206
Epoch 34/200
 - 0s - loss: 0.1205 - mean_squared_error: 0.1205
Epoch 35/200
 - 0s - loss: 0.1204 - mean_squared_error: 0.1204
Epoch 36/200
 - 0s - loss: 0.1202 - mean_squared_error: 0.1202
Epoch 37/200
 - 0s - loss: 0.1201 - mean_squared_error: 0.1201
Epoch 38/200
 - 0s - loss: 0.1200 - mean_squared_error: 0.1200
Epoch 39/200
 - 0s - loss: 0.1199 - mean_squared_error:

 - 0s - loss: 0.0775 - mean_squared_error: 0.0775
Epoch 154/200
 - 0s - loss: 0.0768 - mean_squared_error: 0.0768
Epoch 155/200
 - 0s - loss: 0.0762 - mean_squared_error: 0.0762
Epoch 156/200
 - 0s - loss: 0.0754 - mean_squared_error: 0.0754
Epoch 157/200
 - 0s - loss: 0.0748 - mean_squared_error: 0.0748
Epoch 158/200
 - 0s - loss: 0.0740 - mean_squared_error: 0.0740
Epoch 159/200
 - 0s - loss: 0.0733 - mean_squared_error: 0.0733
Epoch 160/200
 - 0s - loss: 0.0726 - mean_squared_error: 0.0726
Epoch 161/200
 - 0s - loss: 0.0719 - mean_squared_error: 0.0719
Epoch 162/200
 - 0s - loss: 0.0711 - mean_squared_error: 0.0711
Epoch 163/200
 - 0s - loss: 0.0704 - mean_squared_error: 0.0704
Epoch 164/200
 - 0s - loss: 0.0697 - mean_squared_error: 0.0697
Epoch 165/200
 - 0s - loss: 0.0689 - mean_squared_error: 0.0689
Epoch 166/200
 - 0s - loss: 0.0681 - mean_squared_error: 0.0681
Epoch 167/200
 - 0s - loss: 0.0674 - mean_squared_error: 0.0674
Epoch 168/200
 - 0s - loss: 0.0667 - mean_squared_erro

In [ ]:
# plotting the dishwasher load for every 24hrs
i = 0
while(i <= x.shape[0]):
    s = np.array(read1.values[i:i+1439,0])
    k = [t[-9:-7]+t[-6:-4] for t in s ]
    k = [int(p) for p in k]
    l = read.values[i:i+1439,0]
    plt.grid(True)
    plt.plot(k,l)
    plt.show()
    i+=1440